# nb20230220_083900_plot_session_bugfix.ipynb

Bsed on
`notebooks\noncanonical\nb20230212_195300_reaching_go_spout_incr_break2_nov22_kms058_060_061.ipynb`

```
Unexpected err=ValueError('max() arg is an empty sequence'), type(err)=<class 'ValueError'>, for kms058-2023-02-19-190600_reaching_go_spout_incr_break2_nov22.smrx
```


```bash
jupyter nbconvert "D:\OneDrive - Nexus365\Private_Dropbox\Projects\trialexp\notebooks\noncanonical\nb20230212_195300_reaching_go_spout_incr_break2_nov22_kms058_060_061.ipynb" --to="python" --output-dir="D:\OneDrive - Nexus365\Private_Dropbox\Projects\trialexp\notebooks\noncanonical" --output="nb20230212_195300_reaching_go_spout_incr_break2_nov22_kms058_060_061"
```

Quick analysis of instrumental reaching

In [4]:
# allow for automatic reloading of classes and function when updating the code
%load_ext autoreload
%autoreload 2 

# Import Session and Experiment class with helper functions
from trialexp.process.data_import import *



### Variables

In [5]:
import pandas as pd

trial_window = [-2000, 6000] # in ms

# time limit around trigger to perform an event
# determine successful trials
timelim = [0, 2000] # in ms

# Digital channel nb of the pyphotometry device
# on which rsync signal is sent (from pycontrol device)
rsync_chan = 2

basefolder, _ = os.path.split(os.path.split(os.getcwd())[0])

# These must be absolute paths
# use this to use within package tasks files (in params)
tasksfile = os.path.join(basefolder,'params\\tasks_params.csv')
# use this to put a local full path
#tasksfile = -r'C:/.../tasks_params.csv' 

# photometry_dir = r'\\ettin\Magill_Lab\Julien\Data\head-fixed\test_folder\photometry'
photometry_dir = r'\\ettin\Magill_Lab\Julien\Data\head-fixed\kms_pyphotometry'
video_dir = r'\\ettin\Magill_Lab\Julien\Data\head-fixed\videos'

In [6]:
tasks = pd.read_csv(tasksfile, usecols=[1, 2, 3, 4], index_col=False)
tasks


,task,triggers,events,conditions
0,lick_go_nogo,go; nogo,bar; bar_off; spout,free_reward
1,lick_go_nogo_unconditionned,go; nogo,bar; bar_off; spout,free_reward
2,lick_go_nogo_unconditionned_opto,go; nogo,bar; bar_off; spout,free_reward
3,reaching_go_nogo,CS_Go; CS_NoGo,bar; bar_off; spout,error bar_off; reward bar_off; error free; rew...
4,reaching_go_nogo_jc,CS_plus; CS_minus,bar; bar_off; spout,error bar_off; reward bar_off; error free; rew...
5,reaching_go_nogo_opto_continuous,CS_Go; CS_NoGo,bar; bar_off; spout,s_nogo_sham; s_nogo_cs_onset; s_go_sham; s_go_...
6,reaching_go_nogo_opto_sinusoid,CS_Go; CS_NoGo,bar; bar_off; spout,s_nogo_sham; s_nogo_cs_onset_2; s_nogo_cs_onse...
7,reaching_go_nogo_opto_sinusoid_spout,CS_Go; CS_NoGo,bar; bar_off; spout,s_nogo_sham; s_nogo_cs_onset_2; s_nogo_cs_onse...
8,reaching_go_nogo_reversal,CS_Go; CS_NoGo,bar; bar_off; spout,error bar_off; reward bar_off; error free; rew...
9,reaching_go_nogo_reversal_incentive,CS_Go; CS_NoGo,bar; bar_off; spout,error bar_off; reward bar_off; error free; rew...


### Create an experiment object


In [7]:
# Folder of a full experimental batch, all animals included

# Enter absolute path like this
# pycontrol_files_path = r'T:\Data\head-fixed\test_folder\pycontrol'

# or this if you want to use data from the sample_data folder within the package
#pycontrol_files_path = os.path.join(basefolder, 'sample_data/pycontrol')
pycontrol_files_path = r'\\ettin\Magill_Lab\Julien\Data\head-fixed\pycontrol\reaching_go_spout_incr_break2_nov22'

# Load all raw text sessions in the indicated folder or a sessions.pkl file
# if already existing in folder_path
exp_cohort = Experiment(pycontrol_files_path, update = True) #TODO

# Only use if the Experiment cohort as been processed by trials before
# TODO: assess whether this can be removed or not
exp_cohort.by_trial = True


smrx_folder_path = r'\\ettin\Magill_Lab\Julien\Data\head-fixed\pycontrol\reaching_go_spout_incr_break2_nov22\processed'


## Select sessions

In [8]:
import datetime
ss = exp_cohort.sessions

ss_ = [this_ss for this_ss in ss
       if (this_ss.subject_ID in [58])
       and (this_ss.experiment_name == 'reaching_go_spout_incr_break2_nov22')
       and (this_ss.datetime.date() >= datetime.date(2023, 2, 19))]
ss_


In [9]:
exp_cohort.sessions = ss_

In [10]:
ss_


In [11]:
ss_[0].datetime.date()


datetime.date(2023, 2, 19)

# SLOW 3m

In [12]:
exp_cohort.subject_IDs


[58]

In [13]:
# Many combinations possible
conditions_dict0 = {'trigger': 'hold_for_water', 'valid': True}


# Aggregate all condition dictionaries in a list
condition_list = [conditions_dict0]
# Aliases for conditions
cond_aliases = [
    'any_trial',
]

# Groups as a list of lists
groups = None

# right_handed = [281]
# groups = [[280, 282, 299, 300, 301],\
#     [284, 285, 296, 297, 306, 307]]
# Window to exctract (in ms)


In [14]:
exp_cohort.sessions[0].times.keys()

dict_keys(['short_break', 'busy_win', 'button_release', 'US_delay_timer', 'CS_offset_timer', 'bar_off', 'hold_timer', 'spout_off', 'busy_win_timer', 'spout_ana', 'spout_ana_off', 'bar', 'US_end_timer', 'spout', 'button_press', 'block_timer', 'rsync', 'finish_timer'])

# Session plot 

I realised that this plot can never tell if a water drop was triggered by bar_off or spout.


In [15]:
exp_cohort.sessions[0].print_lines[0:30]

['2260 water_on',
 '2475 water_on',
 '3905 water_on',
 '4003 T:2001 this trial> Success:O, 3 spouts\n#1: success:1, free:0, max allowed 6.0, next break 0.36 s, ---------O',
 '5338 water_on',
 '6370 T:4368 this trial> Success:O, 1 spouts\n#2: success:2, free:0, max allowed 6.0, next break 0.40 s, --------OO',
 '7235 water_on',
 '8775 T:6773 this trial> Success:O, 1 spouts\n#3: success:3, free:0, max allowed 6.0, next break 0.44 s, -------OOO',
 '9524 water_on',
 '10475 water_on',
 '11219 T:9217 this trial> Success:O, 2 spouts\n#4: success:4, free:0, max allowed 6.0, next break 0.48 s, ------OOOO',
 '11925 water_on',
 '13472 water_on',
 '13702 T:11700 this trial> Success:O, 2 spouts\n#5: success:5, free:0, max allowed 6.0, next break 0.52 s, -----OOOOO',
 '15470 water_on',
 '16226 T:14224 this trial> Success:O, 1 spouts\n#6: success:6, free:0, max allowed 6.0, next break 0.56 s, ----OOOOOO',
 '17504 water_on',
 '18790 T:16788 this trial> Success:O, 1 spouts\n#7: success:7, free:0, max al

In [16]:
import re

# re.match('abc ','abc de')

# expr = '^\d+(?= ' + '.?Timestamp' + ')'
# a = [re.match(expr, L) for L in exp_cohort.sessions[0].print_lines if re.match(expr , L) is not None]
# int(a[0].group(0))

In [17]:
for ss in exp_cohort.sessions:
    smrxname = re.sub('\.txt', f'_{ss.task_name}.smrx', ss.file_name)
    print(smrxname)


kms058-2023-02-19-184343_reaching_go_spout_incr_break2_nov22.smrx
kms058-2023-02-19-190600_reaching_go_spout_incr_break2_nov22.smrx


In [18]:
exp_cohort.sessions[0].print_lines[0]

a = re.sub('\n','',exp_cohort.sessions[0].print_lines[0])

print(a)

2260 water_on



# export

In [22]:
keys = [
        'button_press', 'bar', 'bar_off', 'spout', 'US_delay_timer', 'CS_offset_timer']

state_def = [
    {'name': 'busy_win',    'onset': 'busy_win',    'offset': 'short_break'},
    {'name': 'short_break', 'onset': 'short_break', 'offset': 'busy_win'}]

summary_df = pd.DataFrame()

for ss in exp_cohort.sessions:

    file_name = os.path.split(ss.file_name)
    file_name_ = re.sub('\.txt',  f'_{ss.task_name}.smrx', file_name[1])
    smrxname = os.path.join(smrx_folder_path, file_name_)
    print(smrxname)


    bw = ss.times['busy_win']
    sp = ss.times['spout']

    x_spout = [this_bw for this_bw in bw for spouts in sp if (
        spouts < this_bw) and (this_bw - spouts < 100)]

    x_bar = [this_bw for this_bw in bw if not any(
        [(spouts < this_bw) and (this_bw - spouts < 100) for spouts in sp])]
        
    event_ms = [{
        'name': 'triggered by spout',
        'time_ms': x_spout
    },
    {
        'name': 'triggered by bar_off',
        'time_ms': x_bar
    }
    ]

    # try:
    ss.plot_session(
        keys, state_def, export_smrx=True, event_ms=event_ms, smrx_filename= smrxname)

    summary_df = pd.concat([summary_df, 
        pd.DataFrame({
            'file':ss.file_name,
            'task':ss.task_name,
            'triggered_by_spout': len(x_spout),
            'triggered_by_bar_off': len(x_bar),
            'reaching_trials': len(bw),
            'trials': len(ss.times['busy_win'])},
            index=[0])
            ],
            ignore_index=True)
    # except Exception as err:
    #     print(f"Unexpected {err=}, {type(err)=}, for {file_name_}")



\\ettin\Magill_Lab\Julien\Data\head-fixed\pycontrol\reaching_go_spout_incr_break2_nov22\processed\kms058-2023-02-19-184343_reaching_go_spout_incr_break2_nov22.smrx


saved \\ettin\Magill_Lab\Julien\Data\head-fixed\pycontrol\reaching_go_spout_incr_break2_nov22\processed\kms058-2023-02-19-184343_reaching_go_spout_incr_break2_nov22.smrx
\\ettin\Magill_Lab\Julien\Data\head-fixed\pycontrol\reaching_go_spout_incr_break2_nov22\processed\kms058-2023-02-19-190600_reaching_go_spout_incr_break2_nov22.smrx


saved \\ettin\Magill_Lab\Julien\Data\head-fixed\pycontrol\reaching_go_spout_incr_break2_nov22\processed\kms058-2023-02-19-190600_reaching_go_spout_incr_break2_nov22.smrx
